<a href="https://colab.research.google.com/github/kstv364/979EdmaVBo/blob/master/hackathon_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install beautifulsoup4 requests cairosvg lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00


In [2]:


import requests
from bs4 import BeautifulSoup
import cairosvg

# ==== 1. Load HTML ====
# If you have the HTML from a webpage:
url = "https://example.com"  # <-- replace with your webpage
response = requests.get(url)
html = response.text

# OR directly use the provided HTML snippet:
html = """
<svg id="hyland_logo" width="232" height="48" viewBox="0 0 232 48" fill="none" xmlns="http://www.w3.org/2000/svg">
  <g clip-path="url(#clip0_3796_22863)">
    <path d="M112.303 16.411V31.7728H108.729V25.4395H99.4351V31.7728H95.8604V16.411H99.4351V22.4024H108.727V16.411H112.303ZM130.796 16.411H134.993L127.349 25.397V31.7728H123.774V25.397L116.108 16.411H120.412L125.614 22.4874L130.794 16.411H130.796ZM138.796 16.411H142.371V28.7144H152.754V31.7728H138.796V16.411ZM168.293 16.411L175.937 31.7728H172.061L170.435 28.4787H162L160.374 31.7728H156.563L164.246 16.411H168.293ZM163.454 25.5246H168.998L166.236 19.9197L163.452 25.5246H163.454ZM195.947 31.7709H192.8L183.206 20.9668V31.7709H179.738V16.4091H183.357L192.479 26.6992V16.4091H195.947V31.7709ZM202 16.4033H210.014C215.509 16.4033 218.954 19.4173 218.954 24.1315C218.954 28.8458 215.507 31.8598 210.014 31.8598H202V16.4033ZM209.927 28.7665C213.26 28.7665 215.209 26.9369 215.209 24.1315C215.209 21.3262 213.264 19.4946 209.908 19.4946H205.482V28.7685H209.929L209.927 28.7665ZM224.968 17.122V19.9834H224.137V17.122H222.583V16.411H226.517V17.122H224.968ZM229.307 19.9834L228.096 17.2322V19.9834H227.284V16.411H228.539L229.653 18.9691L230.764 16.411H232V19.9834H231.194V17.2322L229.978 19.9834H229.307Z" fill="white"></path>
    <path d="M43.3629 0L29.6323 23.9923H45.534L59.2645 0H43.3629Z" fill="#6E33FF"></path>
    <path d="M13.7305 23.9961L0 47.9864H15.9017L29.6322 23.9961H13.7305Z" fill="#f1cb61"></path>
    <path d="M45.5494 24.0078L31.8188 48.0001H47.7224L61.451 24.0078H45.5494Z" fill="#52A1FF"></path>
    <path d="M75.1836 0.015625L61.4512 24.0079H77.3548L91.0872 0.015625H75.1836Z" fill="#13eac1"></path>
  </g>
  <defs>
    <clipPath id="clip0_3796_22863">
      <rect width="232" height="48" fill="white"></rect>
    </clipPath>
  </defs>
</svg>
"""

# ==== 2. Extract <svg> ====
soup = BeautifulSoup(html, "lxml")
svg_tag = soup.find("svg", {"id": "hyland_logo"})

if svg_tag:
    svg_code = str(svg_tag)
    with open("logo.svg", "w") as f:
        f.write(svg_code)
    print("✅ SVG extracted and saved as logo.svg")
else:
    print("❌ No logo found")

# ==== 3. Convert SVG → PNG ====
cairosvg.svg2png(bytestring=svg_code.encode("utf-8"), write_to="logo.png")
print("✅ PNG saved as logo.png")


✅ SVG extracted and saved as logo.svg
✅ PNG saved as logo.png


In [1]:
# Colab-ready script to auto-populate CONFIG from a website
!pip install requests beautifulsoup4 cssutils matplotlib pillow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 9.1 MB/s eta 0:00:00


In [12]:
# ===== Full replacement: Robust dataset generator (Colab-ready) =====
# Paste this full cell into Colab and run (it replaces earlier broken helpers).
!pip install --quiet pillow numpy opencv-python-headless tqdm

import os, random, math, csv, shutil, requests, io, re
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import numpy as np
from tqdm import tqdm

# -------------------------------
# CONFIG — edit as needed before running
# -------------------------------
CONFIG = {
    "OUTDIR": "/content/brand_dataset",
    "brand_name": "Hyland Software",
    "brand_primary_hex": "#191F5E",
    "brand_secondary_hex": "#FFFFFF",
    "brand_accent_hex": "#421F99",
    "num_clean_docs": 30,
    "augmentations_per_doc": 5,
    "lora_train_count": 30,
    "image_dpi": 300,
    "image_size": (1024, 1448), # Reduced image size
    "logo_upload_path": "/content/logo.png",   # set to your uploaded logo if present
    "font_upload_path": None,
}

OUTDIR = Path(CONFIG["OUTDIR"])
IMG_W, IMG_H = CONFIG["image_size"] # Update IMG_W and IMG_H

# -------------------------------
# Helpers
# -------------------------------
def prepare_dirs(outdir):
    dirs = {
        "brand_assets": outdir / "brand_assets",
        "clean_docs": outdir / "clean_docs",
        "dirty_docs": outdir / "dirty_docs",
        "lora_train": outdir / "lora_train",
        "demo_pairs": outdir / "demo_pairs",
        "manifests": outdir / "manifests",
    }
    for p in dirs.values():
        p.mkdir(parents=True, exist_ok=True)
    return dirs

def hex_to_rgb(hx, fallback=(0,0,0)):
    if not hx:
        return fallback
    if isinstance(hx, (tuple, list)) and len(hx) == 3:
        try:
            return tuple(int(x) for x in hx)
        except:
            return fallback
    s = str(hx).strip()
    # rgb(r,g,b)
    m = re.match(r'rgb\(\s*(\d{1,3})\s*,\s*(\d{1,3})\s*,\s*(\d{1,3})\s*\)', s)
    if m:
        return (int(m.group(1)), int(m.group(2)), int(m.group(3)))
    if s.startswith("#"):
        s = s[1:]
    if len(s) == 3:
        s = ''.join([c*2 for c in s])
    if len(s) != 6:
        # try comma-separated
        if "," in s:
            parts = [p.strip() for p in s.split(",")]
            if len(parts) == 3:
                try:
                    return (int(parts[0]), int(parts[1]), int(parts[2]))
                except:
                    return fallback
        return fallback
    try:
        return tuple(int(s[i:i+2], 16) for i in (0,2,4))
    except:
        return fallback

def get_system_font_dest(dest_path):
    # Try common system fonts available on Colab
    candidates = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
        "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
        "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        "/usr/share/fonts/truetype/noto/NotoSans-Regular.ttf",
    ]
    for c in candidates:
        if os.path.exists(c):
            try:
                shutil.copy(c, dest_path)
                return dest_path
            except:
                pass
    return None

def download_font_if_requested(font_url, dest):
    # Try to download if URL provided, else fallback to system fonts.
    if not font_url:
        return get_system_font_dest(dest)
    try:
        resp = requests.get(font_url, timeout=20)
        resp.raise_for_status()
        with open(dest, "wb") as f:
            f.write(resp.content)
        return dest
    except Exception:
        # fallback
        return get_system_font_dest(dest)

def safe_text_size(draw, text, font):
    # textbbox preferred; fallback to textsize or font.getsize
    try:
        bbox = draw.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        return (w,h)
    except Exception:
        try:
            return draw.textsize(text, font=font)
        except Exception:
            try:
                return font.getsize(text)
            except Exception:
                return (len(text)*8, 16)

# -------------------------------
# Placeholder logo creator (safe)
# -------------------------------
def create_placeholder_logo(path, brand, size=(900,220), font_path=None):
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    primary = hex_to_rgb(brand.get("primary"), (11,95,255))
    secondary = hex_to_rgb(brand.get("secondary"), (255,255,255))
    img = Image.new("RGBA", size, primary + (255,))
    draw = ImageDraw.Draw(img)
    # select font
    try:
        if font_path and Path(font_path).exists():
            f = ImageFont.truetype(str(font_path), 64)
        else:
            system = get_system_font_dest(str(p.parent/"_tmp_font.ttf"))
            if system:
                f = ImageFont.truetype(system, 64)
                # cleanup tmp later
            else:
                f = ImageFont.load_default()
    except Exception:
        f = ImageFont.load_default()
    text = brand.get("name", "Brand")
    w,h = safe_text_size(draw, text, f)
    draw.text(((size[0]-w)//2, (size[1]-h)//2), text, fill=secondary + (255,), font=f)
    img.save(path)

# -------------------------------
# Generate one clean doc
# -------------------------------
def generate_clean_doc(path, brand, doc_id, font_paths):
    page = Image.new("RGB", (IMG_W, IMG_H), color=hex_to_rgb(brand["secondary"]))
    draw = ImageDraw.Draw(page)
    # fonts
    try:
        font_regular = ImageFont.truetype(font_paths.get("regular"), 28) if font_paths.get("regular") else ImageFont.load_default()
        font_bold = ImageFont.truetype(font_paths.get("bold"), 36) if font_paths.get("bold") else ImageFont.load_default()
        font_header = ImageFont.truetype(font_paths.get("bold"), 48) if font_paths.get("bold") else ImageFont.load_default()
    except Exception:
        font_regular = ImageFont.load_default()
        font_bold = ImageFont.load_default()
        font_header = ImageFont.load_default()

    # header
    header_h = int(220 * (IMG_H / 3508)) # Scale header height
    draw.line((int(60 * (IMG_W / 2480)), int(40 * (IMG_H / 3508)) + header_h, int((IMG_W-60) * (IMG_W / 2480)), int(40 * (IMG_H / 3508)) + header_h), fill=hex_to_rgb(brand["primary"]), width=int(4 * (IMG_W / 2480))) # Scale line thickness

    # logo area
    logo_w, logo_h = int(700 * (IMG_W / 2480)), int(160 * (IMG_H / 3508)) # Scale logo size
    logo_box = (int(80 * (IMG_W / 2480)), int(60 * (IMG_H / 3508)), int((80+logo_w) * (IMG_W / 2480)), int((60+logo_h) * (IMG_H / 3508))) # Scale logo position
    logo_path = DIRS["brand_assets"] / "logo.png"
    if logo_path.exists():
        try:
            logo = Image.open(logo_path).convert("RGBA")
            logo.thumbnail((logo_w, logo_h), Image.LANCZOS)
            page.paste(logo, (logo_box[0], logo_box[1]), logo if logo.mode=="RGBA" else None)
        except Exception:
            draw.rectangle([logo_box], fill=hex_to_rgb(brand["primary"]))
            draw.text((logo_box[0]+int(10 * (IMG_W / 2480)), logo_box[1]+int(20 * (IMG_H / 3508))), brand["name"], fill=hex_to_rgb(brand["secondary"]), font=font_header)
    else:
        draw.rectangle([logo_box], fill=hex_to_rgb(brand["primary"]))
        draw.text((logo_box[0]+int(20 * (IMG_W / 2480)), logo_box[1]+int(40 * (IMG_H / 3508))), brand["name"], fill=hex_to_rgb(brand["secondary"]), font=font_header)

    # company info (right)
    info_x = logo_box[2] + int(40 * (IMG_W / 2480)) # Scale info position
    info_y = logo_box[1]
    draw.text((info_x, info_y), f"{brand['name']}", font=font_bold, fill=hex_to_rgb(brand["primary"]))
    draw.text((info_x, info_y+int(50 * (IMG_H / 3508))), "123, Business Park, City", font=font_regular, fill=(0,0,0))
    draw.text((info_x, info_y+int(90 * (IMG_H / 3508))), "support@company.example | +91 555 1234", font=font_regular, fill=(0,0,0))

    # title + body
    title_y = int(40 * (IMG_H / 3508)) + header_h + int(60 * (IMG_H / 3508)) # Scale title position
    draw.text((int(80 * (IMG_W / 2480)), title_y), f"Invoice / Report #{doc_id:04d}", font=font_bold, fill=(0,0,0))
    body_y = title_y + int(70 * (IMG_H / 3508)) # Scale body position
    margin_l, margin_r = int(80 * (IMG_W / 2480)), IMG_W-int(80 * (IMG_W / 2480)) # Scale margins
    line_h = int(34 * (IMG_H / 3508)) # Scale line height
    lorem = ("Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. "
             "Sed cursus ante dapibus diam. Sed nisi. Nulla quis sem at nibh elementum imperdiet.")
    y = body_y
    for p in range(6):
        words = lorem.split(" ")
        line = ""
        for w in words:
            test_line = (line + " " + w).strip()
            w_sz = safe_text_size(draw, test_line, font_regular)[0]
            if w_sz > (margin_r - margin_l):
                draw.text((margin_l, y), line, font=font_regular, fill=(20,20,20))
                y += line_h
                line = w
            else:
                line = test_line
        if line:
            draw.text((margin_l, y), line, font=font_regular, fill=(20,20,20))
            y += line_h
        y += int(10 * (IMG_H / 3508)) # Scale paragraph spacing

    # simple table
    table_top = y + int(20 * (IMG_H / 3508)) # Scale table position
    col_x = [margin_l, margin_l+int(700 * (IMG_W / 2480)), margin_l+int(1200 * (IMG_W / 2480)), margin_r] # Scale column positions
    row_h = int(70 * (IMG_H / 3508)) # Scale row height
    rows = 6
    draw.rectangle([col_x[0], table_top, col_x[-1], table_top + row_h], fill=hex_to_rgb(brand["primary"]))
    draw.text((col_x[0]+int(10 * (IMG_W / 2480)), table_top+int(18 * (IMG_H / 3508))), "Item", font=font_bold, fill=hex_to_rgb(brand["secondary"]))
    draw.text((col_x[1]+int(10 * (IMG_W / 2480)), table_top+int(18 * (IMG_H / 3508))), "Description", font=font_bold, fill=hex_to_rgb(brand["secondary"]))
    draw.text((col_x[2]+int(10 * (IMG_W / 2480)), table_top+int(18 * (IMG_H / 3508))), "Price", font=font_bold, fill=hex_to_rgb(brand["secondary"]))
    for r in range(1, rows):
        y0 = table_top + r*row_h
        draw.line((col_x[0], y0, col_x[-1], y0), fill=(180,180,180), width=int(2 * (IMG_W / 2480))) # Scale line thickness
        draw.text((col_x[0]+int(10 * (IMG_W / 2480)), y0+int(18 * (IMG_H / 3508))), f"Item {r}", font=font_regular, fill=(0,0,0))
        draw.text((col_x[1]+int(10 * (IMG_W / 2480)), y0+int(18 * (IMG_H / 3508))), "Product description", font=font_regular, fill=(0,0,0))
        draw.text((col_x[2]+int(10 * (IMG_W / 2480)), y0+int(18 * (IMG_H / 3508))), f"₹{1000*r}", font=font_regular, fill=(0,0,0))

    # footer
    footer_h = int(140 * (IMG_H / 3508)) # Scale footer height
    footer_y = IMG_H - footer_h - int(60 * (IMG_H / 3508)) # Scale footer position
    draw.line((int(60 * (IMG_W / 2480)), footer_y, int((IMG_W-60) * (IMG_W / 2480)), footer_y), fill=hex_to_rgb(brand["primary"]), width=int(3 * (IMG_W / 2480))) # Scale line thickness
    draw.text((int(80 * (IMG_W / 2480)), footer_y+int(20 * (IMG_H / 3508))), f"{brand['name']} | Confidential", font=font_regular, fill=(80,80,80))
    draw.text((IMG_W-int(400 * (IMG_W / 2480)), footer_y+int(20 * (IMG_H / 3508))), "Page 1 of 1", font=font_regular, fill=(80,80,80))

    page.save(path, dpi=(CONFIG["image_dpi"], CONFIG["image_dpi"]))

# -------------------------------
# Corruptions
# -------------------------------
def add_gaussian_noise_pil(img_pil, sigma=20):
    img = np.array(img_pil).astype(np.float32)
    noise = np.random.normal(0, sigma, img.shape).astype(np.float32)
    out = img + noise
    out = np.clip(out, 0, 255).astype(np.uint8)
    return Image.fromarray(out)

def add_motion_blur_pil(img_pil, degree=10, angle=0):
    img = np.array(img_pil)
    M = np.zeros((degree, degree))
    M[int((degree-1)/2), :] = np.ones(degree)
    M = Image.fromarray(M).rotate(angle, expand=False)
    M = np.array(M)
    M = M / M.sum() if M.sum() != 0 else M
    blurred = cv2_filter2D_safe(img, M)
    return Image.fromarray(np.clip(blurred, 0, 255).astype(np.uint8))

def cv2_filter2D_safe(img, kernel):
    import cv2
    try:
        return cv2.filter2D(img, -1, kernel)
    except Exception:
        # fallback to PIL gaussian blur approx
        pil = Image.fromarray(img)
        return np.array(pil.filter(ImageFilter.GaussianBlur(radius=1)))

def add_skew_pil(img_pil, max_angle=4):
    angle = random.uniform(-max_angle, max_angle)
    return img_pil.rotate(angle, expand=False, fillcolor=hex_to_rgb(CONFIG["brand_secondary_hex"]))

def add_color_cast_pil(img_pil):
    arr = np.array(img_pil).astype(np.float32)
    cast = np.array([random.uniform(-20,20), random.uniform(-20,20), random.uniform(-20,20)])
    arr = arr + cast
    arr = np.clip(arr, 0, 255).astype(np.uint8)
    return Image.fromarray(arr)

def add_stain_pil(img_pil, intensity=0.5):
    img = img_pil.copy().convert("RGBA")
    overlay = Image.new("RGBA", img.size, (255,255,255,0))
    draw = ImageDraw.Draw(overlay)
    for _ in range(random.randint(1,3)):
        x0 = random.randint(int(200 * (IMG_W / 2480)), IMG_W-int(600 * (IMG_W / 2480))) # Scale stain position
        y0 = random.randint(int(300 * (IMG_H / 3508)), IMG_H-int(1000 * (IMG_H / 3508))) # Scale stain position
        w = random.randint(int(200 * (IMG_W / 2480)), int(700 * (IMG_W / 2480))) # Scale stain size
        h = random.randint(int(100 * (IMG_H / 3508)), int(400 * (IMG_H / 3508))) # Scale stain size
        bbox = [x0, y0, x0+w, y0+h]
        color = (150, 100, 50, int(80*intensity))
        draw.ellipse(bbox, fill=color)
    overlay = overlay.filter(ImageFilter.GaussianBlur(radius=random.randint(int(15 * (IMG_W / 2480)),int(40 * (IMG_W / 2480))))) # Scale blur radius
    combined = Image.alpha_composite(img, overlay).convert("RGB")
    return combined

def add_shadow_pil(img_pil):
    img = img_pil.copy().convert("RGBA")
    shadow = Image.new("RGBA", img.size, (0,0,0,0))
    draw = ImageDraw.Draw(shadow)
    for i in range(int(200 * (IMG_W / 2480))): # Scale loop iterations
        alpha = int(220*(1 - i/int(220 * (IMG_W / 2480)))) # Scale alpha calculation
        draw.ellipse([int(-400 * (IMG_W / 2480))+i*int(5 * (IMG_W / 2480)), IMG_H-int(800 * (IMG_H / 3508))+i*int(3 * (IMG_H / 3508)), int(800 * (IMG_W / 2480))+i*int(5 * (IMG_W / 2480)), IMG_H+int(400 * (IMG_H / 3508))+i*int(3 * (IMG_H / 3508))], fill=(0,0,0,alpha//8)) # Scale ellipse position and size
    shadow = shadow.filter(ImageFilter.GaussianBlur(radius=int(120 * (IMG_W / 2480)))) # Scale blur radius
    combined = Image.alpha_composite(img, shadow).convert("RGB")
    return combined

# -------------------------------
# Manifest writer
# -------------------------------
def write_manifest(manifest_path, rows):
    keys = rows[0].keys() if rows else []
    with open(manifest_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

# -------------------------------
# Main generation
# -------------------------------
def generate_dataset(cfg):
    global DIRS, IMG_W, IMG_H # Declare IMG_W and IMG_H as global
    outdir = Path(cfg["OUTDIR"])
    DIRS = prepare_dirs(outdir)
    print("Directories prepared at:", outdir)
    # fonts
    font_regular_path = None
    font_bold_path = None
    if cfg.get("font_upload_path") and Path(cfg["font_upload_path"]).exists():
        font_regular_path = cfg["font_upload_path"]
        font_bold_path = cfg["font_upload_path"]
    else:
        # try copying a system font into brand_assets
        font_regular_path = get_system_font_dest(DIRS["brand_assets"]/ "DejaVuSans-Regular.ttf")
        font_bold_path = get_system_font_dest(DIRS["brand_assets"]/ "DejaVuSans-Bold.ttf")
    font_paths = {"regular": str(font_regular_path) if font_regular_path else None,
                  "bold": str(font_bold_path) if font_bold_path else None}

    # logo
    if cfg.get("logo_upload_path") and Path(cfg["logo_upload_path"]).exists():
        logo_dest = DIRS["brand_assets"] / "logo.png"
        shutil.copy(cfg["logo_upload_path"], logo_dest)
        print("Using uploaded logo:", cfg["logo_upload_path"])
    else:
        logo_dest = DIRS["brand_assets"] / "logo.png"
        create_placeholder_logo(logo_dest, {"name": cfg.get("brand_name","Brand"), "primary": cfg.get("brand_primary_hex"), "secondary": cfg.get("brand_secondary_hex")}, size=(int(900 * (IMG_W / 2480)),int(220 * (IMG_H / 3508))), font_path=font_paths.get("bold")) # Scale placeholder logo size
        print("Placeholder logo created at:", logo_dest)

    # generate clean docs
    clean_paths = []
    print("Generating clean documents...")
    for i in tqdm(range(1, cfg["num_clean_docs"]+1)):
        outp = DIRS["clean_docs"] / f"clean_{i:03d}.png"
        generate_clean_doc(outp, {"name": cfg.get("brand_name","Brand"), "primary": cfg.get("brand_primary_hex"), "secondary": cfg.get("brand_secondary_hex")}, i, font_paths)
        clean_paths.append(outp)

    # copy first N into lora_train
    lora_train_list = clean_paths[:max(1, min(len(clean_paths), cfg.get("lora_train_count", len(clean_paths))))]
    for p in lora_train_list:
        dst = DIRS["lora_train"] / p.name
        shutil.copy(p, dst)

    # generate corrupted variants & demo pairs
    rows = []
    print("Generating corrupted variants...")
    for clean_p in tqdm(clean_paths):
        for k in range(cfg["augmentations_per_doc"]):
            outp = DIRS["dirty_docs"] / f"{clean_p.stem}_aug{k}.png"
            img = Image.open(clean_p).convert("RGB")
            if random.random() < 0.75:
                img = add_gaussian_noise_pil(img, sigma=random.uniform(6,40))
            if random.random() < 0.6:
                img = add_motion_blur_pil(img, degree=random.choice([5,9,15]), angle=random.uniform(-40,40))
            if random.random() < 0.7:
                img = add_skew_pil(img, max_angle=6)
            if random.random() < 0.5:
                img = add_color_cast_pil(img)
            if random.random() < 0.45:
                img = add_stain_pil(img, intensity=random.uniform(0.3,1.0))
            if random.random() < 0.45:
                img = add_shadow_pil(img)
            if random.random() < 0.6:
                img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1,1.4)))
            img.save(outp, dpi=(cfg["image_dpi"], cfg["image_dpi"]))
            # demo pairs subset
            if random.random() < 0.4 and len(list(DIRS["demo_pairs"].glob("*_clean.png"))) < 40:
                demo_clean = DIRS["demo_pairs"] / f"{clean_p.stem}_clean.png"
                demo_dirty = DIRS["demo_pairs"] / f"{clean_p.stem}_dirty.png"
                if not demo_clean.exists():
                    shutil.copy(clean_p, demo_clean)
                shutil.copy(outp, demo_dirty)
            rows.append({"clean": str(clean_p), "dirty": str(outp)})

    manifest_path = DIRS["manifests"] / "manifest.csv"
    write_manifest(manifest_path, rows)
    print("Dataset generation complete.")
    print("Summary:")
    print("  clean docs:", len(clean_paths))
    print("  dirty variants:", len(rows))
    print("  lora_train examples:", len(list(DIRS["lora_train"].glob("*.png"))))
    print("  demo pairs:", len(list(DIRS["demo_pairs"].glob("*_clean.png"))))


# -------------------------------
# Run generator
# -------------------------------
if __name__ == "__main__":
    generate_dataset(CONFIG)
    print("\nDone. Inspect the dataset at:", CONFIG["OUTDIR"])
    print("Tip: open /content/brand_dataset/demo_pairs to pick showcase images.")

Directories prepared at: /content/brand_dataset
Using uploaded logo: /content/logo.png
Generating clean documents...


100%|██████████| 30/30 [00:24<00:00,  1.22it/s]


Generating corrupted variants...


100%|██████████| 30/30 [14:57<00:00, 29.93s/it]

Dataset generation complete.
Summary:
  clean docs: 30
  dirty variants: 150
  lora_train examples: 30
  demo pairs: 30

Done. Inspect the dataset at: /content/brand_dataset
Tip: open /content/brand_dataset/demo_pairs to pick showcase images.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Colab cell (bash)
# Install dependencies
!pip install -q git+https://github.com/huggingface/diffusers.git transformers accelerate safetensors \
  xformers==0.0.27  # optional but speeds up training if available

# other utilities
!pip install -q datasets huggingface_hub accelerate matplotlib

# clone diffusers examples (to use the DreamBooth LoRA script)
!git clone https://github.com/huggingface/diffusers.git /content/diffusers_repo


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 842.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB

In [6]:
!nvidia-smi

Mon Aug 25 20:15:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.5 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.23.0+cu126
    Uninstalling torchvision-0.23.0+cu126:
      Successfully uninstalled torchvision-0.23.0+cu126
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.8.0+cu126
    Uninstalling torchaudio-2.8.0+cu126:
      Successfully uninstalled torchaudio-2.8.0+cu126


In [10]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.9 MB/s eta 0:00:00


In [11]:
# Colab cell (bash) — edit these paths/values
PRETRAINED_MODEL="runwayml/stable-diffusion-v1-5"   # base SD checkpoint (HF)
INSTANCE_DIR="/content/drive/MyDrive/brand_dataset/lora_train"   # your images (clean branded pages)
OUTPUT_DIR="/content/brand_lora"                   # loRA output
INSTANCE_PROMPT="a photo of <brand_style> document"  # class prompt for your brand token
TRAIN_RES=512
BATCH=1
STEPS=800   # reduce for faster runs, increase for better quality
LR=1e-4

# Create output dir
!mkdir -p $OUTPUT_DIR

# Run training script from diffusers examples
!accelerate launch /content/diffusers_repo/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$PRETRAINED_MODEL \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=$TRAIN_RES \
  --train_batch_size=$BATCH \
  --max_train_steps=$STEPS \
  --learning_rate=$LR \
  --mixed_precision="fp16" \
  --use_8bit_adam


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-08-25 20:23:30.359206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756153410.379182    6064 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756153410.385462    6064 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756153410.401092    6064 computation_plac

In [13]:
# Colab cell (bash)
!pip install -q paddlepaddle paddleocr pdf2image pypdf2 reportlab gradio opencv-python-headless diffusers transformers accelerate safetensors
!apt-get update -qq && apt-get install -y -qq poppler-utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.5/187.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB

In [ ]:
# Colab cell (python)
import os, io, tempfile, math, shutil
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
import torch
import gradio as gr
from pdf2image import convert_from_path
from paddleocr import PaddleOCR
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image

# ---------------- Config - EDIT if needed ----------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_SD = "runwayml/stable-diffusion-v1-5"
CONTROLNET_MODEL = "lllyasviel/sd-controlnet-canny"
LOADED_LORA_DIR = "/content/brand_lora"   # set to output dir from Part A or None
BRAND_TOKEN = "<brand_style>"             # the token you used in training (instance_prompt)
PIPELINE_STEPS = 18

# ---------------- OCR ----------------
ocr = PaddleOCR(use_angle_cls=True, lang="en")

# ---------------- Utils ----------------
def pdf_to_images(file_path):
    images = convert_from_path(file_path, dpi=150)
    imgs = [im.convert("RGB") for im in images]
    return imgs

def deskew_image(pil_img):
    # convert to grayscale and compute skew via Hough or moments
    img = np.array(pil_img.convert("L"))
    coords = np.column_stack(np.where(img < 250))
    if coords.shape[0] < 10:
        return pil_img
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = img.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(np.array(pil_img), M, (w, h), borderValue=(255,255,255))
    return Image.fromarray(rotated)

def denoise_image(pil_img):
    arr = np.array(pil_img)
    den = cv2.fastNlMeansDenoisingColored(arr, None, 10, 10, 7, 21)
    return Image.fromarray(den)

def build_canny(pil_img):
    arr = np.array(pil_img.convert("L"))
    edges = cv2.Canny(arr, 100, 200)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges_rgb)

def run_ocr(pil_img):
    # returns list of (text, box)
    tmp = "/tmp/tmp_ocr.png"
    pil_img.save(tmp)
    res = ocr.ocr(tmp)
    outputs = []
    if len(res) > 0:
        for line in res[0]:
            txt = line[1][0]
            box = line[0]  # list of 4 points
            outputs.append((txt, box))
    return outputs

def overlay_ocr_on_image(pil_img, ocr_results, font_path=None):
    img = pil_img.copy()
    draw = ImageDraw.Draw(img)
    if font_path and Path(font_path).exists():
        try:
            font = ImageFont.truetype(font_path, 12)
        except:
            font = ImageFont.load_default()
    else:
        font = ImageFont.load_default()
    for text, box in ocr_results:
        # compute top-left of bbox
        xs = [p[0] for p in box]; ys=[p[1] for p in box]
        x, y = int(min(xs)), int(min(ys))
        draw.text((x, y), text, fill=(0,0,0), font=font)
    return img

# ---------------- Load models ----------------
print("Device:", DEVICE)
print("Loading ControlNet...")
controlnet = ControlNetModel.from_pretrained(CONTROLNET_MODEL, torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32)
print("Loading SD pipeline (this may take a while)...")
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_SD, controlnet=controlnet, torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
if DEVICE == "cuda":
    pipe = pipe.to("cuda")
else:
    pipe = pipe.to("cpu")

# Load LoRA weights if present
if LOADED_LORA_DIR and os.path.exists(LOADED_LORA_DIR):
    print("Loading LoRA from:", LOADED_LORA_DIR)
    try:
        # this expects the diffusers LoRA output format (attn_procs or adapter_model.safetensors saved by script)
        pipe.unet.load_attn_procs(LOADED_LORA_DIR)
        print("LoRA loaded into UNet.")
    except Exception as e:
        print("Could not load LoRA via load_attn_procs():", e)
        print("If you used a different LoRA export, adapt loading accordingly. Proceeding without LoRA.")
else:
    print("No LoRA dir found — running without LoRA.")

# ---------------- Inference wrapper ----------------
def resize_for_inference(pil_img, max_size=1024):
    w, h = pil_img.size
    scale = max_size / max(w, h)
    if scale < 1.0:  # only shrink, never upscale
        new_w, new_h = int(w * scale), int(h * scale)
        pil_img = pil_img.resize((new_w, new_h), Image.LANCZOS)
    return pil_img

def process_page(pil_img, prompt_extra=""):
    # 0) resize to speed up
    pil_img = resize_for_inference(pil_img, max_size=1024)

    # 1) deskew + denoise
    sk = deskew_image(pil_img)
    dn = denoise_image(sk)

    # 2) OCR (we'll overlay later)
    ocr_res = run_ocr(dn)

    # 3) ControlNet input (Canny)
    canny = build_canny(dn)

    # 4) SD+ControlNet run
    prompt = f"Document in {BRAND_TOKEN} style {prompt_extra}".strip()
    generator = None
    if torch.cuda.is_available():
        generator = torch.Generator(device="cuda").manual_seed(42)
    with torch.autocast("cuda") if torch.cuda.is_available() else torch.no_grad():
        out = pipe(
            prompt,
            num_inference_steps=PIPELINE_STEPS,
            image=canny,
            generator=generator
        ).images[0]

    # 5) overlay OCR text (simple)
    final = overlay_ocr_on_image(out, ocr_res)

    return dn, out, final, ocr_res


# ---------------- Gradio app ----------------
def run_pipeline_upload(uploaded_file):
    # uploaded_file is a tempfile-like object with .name
    fname = uploaded_file.name
    ext = Path(fname).suffix.lower()
    pages = []
    if ext == ".pdf":
        pages = pdf_to_images(fname)
    else:
        pages = [Image.open(fname).convert("RGB")]

    out_pairs = []
    for p in pages:
        orig, styled, final, ocr_res = process_page(p)
        # convert to thumbnails for display (optional)
        out_pairs.append((orig.resize((512, int(512 * orig.height / orig.width))),
                          final.resize((512, int(512 * final.height / final.width)))))
    # Gradio gallery expects images in list form
    # Flatten pairs into list of images in sequence orig, final, orig2, final2, ...
    flat = []
    for a,b in out_pairs:
        flat.append(a); flat.append(b)
    return flat

# Build Gradio UI
title = "ECM Document Brand Restyler (PoC)"
description = "Upload a PDF or image. The app OCRs the page, cleans via ControlNet+SD, overlays OCR text, and shows original vs styled."

with gr.Blocks() as demo:
    gr.Markdown(f"# {title}")
    gr.Markdown(description)
    file_input = gr.File(label="Upload PDF or Image", file_count="single", type="filepath", file_types=[".pdf",".png",".jpg",".jpeg"])
    run_btn = gr.Button("Process")
    gallery = gr.Gallery(
        label="Original (L) / Restyled + OCR overlay (R)",
        columns=2,       # replaces grid
        height="auto"    # valid in >=4.0
    )

    run_btn.click(run_pipeline_upload, inputs=file_input, outputs=gallery)

# Launch
demo.launch(server_name="0.0.0.0", share=True, debug=True)


/tmp/ipython-input-1912908458.py:23: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang="en")
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('en_PP-OCRv5_mobile_rec', None)
Using official model (en_PP-OCRv5_mobile_rec), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Device: cuda
Loading ControlNet...
Loading SD pipeline (this may take a while)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]